<a href="https://colab.research.google.com/github/MoheEdeen/Deepfake-Detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("test-safwah")

test-safwah
